In [1]:
import intake
import easygems.healpix as egh
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
import glob
import logging
import os
import pathlib
import dask
dask.config.set(**{'array.slicing.split_large_chunks': True})

outdir = pathlib.Path(f'/work/bb1153/b382635/plots/tracked_results_2025/dataset_paper/results_data/acp_submission/')
os.makedirs(outdir, exist_ok=True)

In [2]:
# load data
tracks_path = '/work/bb1153/b382635/data/final_tracks/updraft_ice_only/amazon/'
features_path = '/work/bb1153/b382635/data/final_tracks/amazon/'
tracks = glob.glob(tracks_path + '*/*system_tracks_linked_proc.nc')
pre_proc_tracks = glob.glob(tracks_path + '*/*system_tracks_linked.nc')
updrafts = glob.glob(features_path + '*/*updraft/segmented_mask.nc')
ice = glob.glob(features_path + '*/*ice/segmented_mask.nc')

tracks = xr.open_mfdataset(tracks)
pp_tracks = xr.open_mfdataset(pre_proc_tracks)
updrafts = xr.open_mfdataset(updrafts)
ice = xr.open_mfdataset(ice)

# to AMT time
tracks['time'] = tracks.time + pd.Timedelta(hours=-4)
pp_tracks['time'] = pp_tracks.time + pd.Timedelta(hours=-4)
updrafts['time'] = updrafts.time + pd.Timedelta(hours=-4)
ice['time'] = ice.time + pd.Timedelta(hours=-4)

In [3]:
# get area covered...

def get_area_covered(ds):
    grid_spacing = 11000 # m
    area = (ds>0).sum(('lat','lon')) * (grid_spacing**2)
    diurnal_mean = area.groupby(area.time.dt.hour).mean()
    return diurnal_mean

area_ds = xr.Dataset({'updrafts': get_area_covered(updrafts.feature>0),
                      'ice': get_area_covered(ice.feature>0),
                      'tracked_updrafts': get_area_covered(pp_tracks.u_tracks>0),
                      'tracked_ice': get_area_covered(pp_tracks.i_tracks>0),
                      'tracked_system': get_area_covered(tracks.system>0),
                      'dcc_ice': get_area_covered(tracks.i_tracks.where(tracks.system>0)>0),
                      'dcc_updraft': get_area_covered(tracks.u_tracks.where(tracks.system>0)>0),
                     })
area_ds = area_ds.compute() # force compute (takes >10 mins)

KeyboardInterrupt: 

In [ ]:
area_ds.tracked_ice.plot()

In [ ]:
area_ds.to_netcdf(outdir / "what_got_tracked.nc")